# Imports and Setups

In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 11.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


Lendo o Arquivo

In [2]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [3]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()


In [4]:
num_gen = 5
perc = 5
i=5

In [5]:
arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/base_tweet_sentiment_'+str(perc)+'_original'+str(i)+'.csv'
base_cru_1p = pd.read_csv(arq, encoding='utf-8', sep=';', decimal = ',')

In [6]:
base_cru_1p.head()

,texto,label
0,Really excited about Apple Watches collab with...,2
1,"""Cameron says the UK is prepared to veto the E...",1
2,Saying jumbo shrimp on the menu when it is CLE...,1
3,"""tomorrow at 9, I'll be fangirling over the tv...",2
4,sa-ANNE ka pupunta bukas? CURTIS video be the ...,1


In [7]:
def cria_prompt(ordem,synt):
  prompt = ordem

  prompt += 'Tweet: '+ synt+'\nResult: '
  return prompt

In [8]:
inicio = """Rewrite the following tweets 5 times to generate 5 different paraphrases  from original tweet.
Return in Python list like this ["paraphrase1","paraphrase2","paraphrase3","paraphrase4","paraphrase5"] as example.

Tweet:What are the best places to see in New York?
Result:["What are some must-see places in New York?","Can you suggest some must-see spots in New York?","Where should one go to experience the best NYC has to offer?","Which places should I visit in New York?","What are the top destinations to explore in New York?"]
###
"""
base_cru_1p['prompt'] = base_cru_1p.progress_apply(lambda x: cria_prompt(inicio, x.texto), axis=1)
base_cru_1p.head()

100%|██████████| 210/210 [00:00<00:00, 33979.01it/s]


,texto,label,prompt
0,Really excited about Apple Watches collab with...,2,Rewrite the following tweets 5 times to genera...
1,"""Cameron says the UK is prepared to veto the E...",1,Rewrite the following tweets 5 times to genera...
2,Saying jumbo shrimp on the menu when it is CLE...,1,Rewrite the following tweets 5 times to genera...
3,"""tomorrow at 9, I'll be fangirling over the tv...",2,Rewrite the following tweets 5 times to genera...
4,sa-ANNE ka pupunta bukas? CURTIS video be the ...,1,Rewrite the following tweets 5 times to genera...


In [9]:
base_cru_1p.texto.iloc[90]

'Report: Steelers can\\u2019t find hotel\\u002c to travel Sun.: The Pittsburgh Steelers will fly to New York only hours before playing the Giants ...'

In [10]:
print(base_cru_1p.prompt.iloc[90])

Rewrite the following tweets 5 times to generate 5 different paraphrases  from original tweet.
Return in Python list like this ["paraphrase1","paraphrase2","paraphrase3","paraphrase4","paraphrase5"] as example.

Tweet:What are the best places to see in New York?
Result:["What are some must-see places in New York?","Can you suggest some must-see spots in New York?","Where should one go to experience the best NYC has to offer?","Which places should I visit in New York?","What are the top destinations to explore in New York?"]
###
Tweet: Report: Steelers can\u2019t find hotel\u002c to travel Sun.: The Pittsburgh Steelers will fly to New York only hours before playing the Giants ...
Result: 


In [11]:
n_labels = base_cru_1p.label.nunique()
n_labels

3

In [12]:
! huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' cre

In [13]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "meta-llama/Llama-2-7b-chat-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model_4bit = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
prompt = """Rewrite the following tweets 5 times to generate 5 different paraphrases  from original tweet.
Return in Python list like this ["paraphrase1","paraphrase2","paraphrase3","paraphrase4","paraphrase5"] as example.

Tweet:What are the best places to see in New York?
Result:["What are some must-see places in New York?","Can you suggest some must-see spots in New York?","Where should one go to experience the best NYC has to offer?","Which places should I visit in New York?","What are the top destinations to explore in New York?"]
###
Tweet: The Quarterback' wrecks me every time..
Result: """

In [15]:
def gera_text(modelo, tokens, dev, ppt, temperatura, max_tokens):
  inputs = tokens(ppt, return_tensors="pt").to(dev)
  outputs = modelo.generate(**inputs, max_new_tokens=max_tokens, temperature=temperatura, do_sample =True)
  result = tokenizer.decode(outputs[0], skip_special_tokens=True)
  return result

In [16]:
device = "cuda:0"
print(gera_text(model_4bit, tokenizer, device, prompt, 0.1, 400))

Rewrite the following tweets 5 times to generate 5 different paraphrases  from original tweet.
Return in Python list like this ["paraphrase1","paraphrase2","paraphrase3","paraphrase4","paraphrase5"] as example.

Tweet:What are the best places to see in New York?
Result:["What are some must-see places in New York?","Can you suggest some must-see spots in New York?","Where should one go to experience the best NYC has to offer?","Which places should I visit in New York?","What are the top destinations to explore in New York?"]
###
Tweet: The Quarterback' wrecks me every time..
Result:  ["The Quarterback's impact never fails to move me...", "The Quarterback's brilliance always leaves me in awe...", "The Quarterback's talent is unmatched and always impresses me...", "The Quarterback's greatness never ceases to amaze me...", "The Quarterback's talent is unparalleled and always astounds me..."]





































































































In [17]:
base_cru_1p['generated'] = base_cru_1p['prompt'].progress_apply(lambda x: gera_text(model_4bit, tokenizer, device, x, 0.1, 400))
base_cru_1p.head()

100%|██████████| 210/210 [1:50:42<00:00, 31.63s/it]


,texto,label,prompt,generated
0,Really excited about Apple Watches collab with...,2,Rewrite the following tweets 5 times to genera...,Rewrite the following tweets 5 times to genera...
1,"""Cameron says the UK is prepared to veto the E...",1,Rewrite the following tweets 5 times to genera...,Rewrite the following tweets 5 times to genera...
2,Saying jumbo shrimp on the menu when it is CLE...,1,Rewrite the following tweets 5 times to genera...,Rewrite the following tweets 5 times to genera...
3,"""tomorrow at 9, I'll be fangirling over the tv...",2,Rewrite the following tweets 5 times to genera...,Rewrite the following tweets 5 times to genera...
4,sa-ANNE ka pupunta bukas? CURTIS video be the ...,1,Rewrite the following tweets 5 times to genera...,Rewrite the following tweets 5 times to genera...


In [18]:
base_cru_1p.head()

,texto,label,prompt,generated
0,Really excited about Apple Watches collab with...,2,Rewrite the following tweets 5 times to genera...,Rewrite the following tweets 5 times to genera...
1,"""Cameron says the UK is prepared to veto the E...",1,Rewrite the following tweets 5 times to genera...,Rewrite the following tweets 5 times to genera...
2,Saying jumbo shrimp on the menu when it is CLE...,1,Rewrite the following tweets 5 times to genera...,Rewrite the following tweets 5 times to genera...
3,"""tomorrow at 9, I'll be fangirling over the tv...",2,Rewrite the following tweets 5 times to genera...,Rewrite the following tweets 5 times to genera...
4,sa-ANNE ka pupunta bukas? CURTIS video be the ...,1,Rewrite the following tweets 5 times to genera...,Rewrite the following tweets 5 times to genera...


In [19]:
import ast
def transforme_list(x):
  try:
    resp = ast.literal_eval(x.split("Result:")[2].replace("\n", "").strip())
  except:
    resp = []
  return resp

In [20]:
print(base_cru_1p.generated.iloc[4])

Rewrite the following tweets 5 times to generate 5 different paraphrases  from original tweet.
Return in Python list like this ["paraphrase1","paraphrase2","paraphrase3","paraphrase4","paraphrase5"] as example.

Tweet:What are the best places to see in New York?
Result:["What are some must-see places in New York?","Can you suggest some must-see spots in New York?","Where should one go to experience the best NYC has to offer?","Which places should I visit in New York?","What are the top destinations to explore in New York?"]
###
Tweet: sa-ANNE ka pupunta bukas? CURTIS video be the reason why you go to #SOLTURA tomorrow? Watch it to find out!!!
Result:  ["Will you be heading to SOLTURA tomorrow? CURTIS video might be the reason why? Watch it to find out!","Are you planning to visit SOLTURA tomorrow? CURTIS video might be the reason why you should! Watch it to find out!","Are you going to SOLTURA tomorrow? CURTIS video might be the reason why you'll want to! Watch it to find out!","Will y

In [21]:
base_cru_1p.generated.iloc[0].split("Result:")[2].replace("\n", "").strip()

'["I\'m thrilled about the Apple Watch collaboration with #Hermes - it\'s going to be stunning! The new models with the latest OS update - everything will be available on September 16.","I\'m beyond excited about the Apple Watch Hermes collaboration - it\'s going to be gorgeous! The new models with the latest OS update will be available on September 16.","I\'m really looking forward to the Apple Watch Hermes collaboration - it\'s going to be lovely! The new models with the latest OS update will be available on September 16.","I\'m super excited about the Apple Watch Hermes collaboration - it\'s going to be amazing! The new models with the latest OS update will be available on September 16.","I\'m incredibly excited about the Apple Watch Hermes collaboration - it\'s going to be breathtaking! The new models with the latest OS update will be available on September 16."]'

In [22]:
print(ast.literal_eval(base_cru_1p.generated.iloc[3].split("Result:")[2].replace("\n", "")))

["I'll be fangirling over the TV show 'Styled To Rock' tomorrow at 9pm! 😍👗🎬", 'Get ready for a fashion extravaganza tomorrow at 9pm! 💃🕺👗', "Tomorrow at 9pm, I'll be losing my mind over 'Styled To Rock' 😂👠🎬", "I'm so excited for tomorrow at 9pm when I'll be glued to my TV for 'Styled To Rock' 😍👗🎬", "Tomorrow at 9pm, I'll be in fashion heaven with the new show 'Styled To Rock' 😘👗🎬"]


In [23]:
base_cru_1p['generated'] = base_cru_1p['generated'].progress_apply(lambda x: transforme_list(x))

100%|██████████| 210/210 [00:00<00:00, 30796.26it/s]


In [24]:
base_cru_1p.head()

,texto,label,prompt,generated
0,Really excited about Apple Watches collab with...,2,Rewrite the following tweets 5 times to genera...,[I'm thrilled about the Apple Watch collaborat...
1,"""Cameron says the UK is prepared to veto the E...",1,Rewrite the following tweets 5 times to genera...,[Cameron claims UK ready to block EU budget: B...
2,Saying jumbo shrimp on the menu when it is CLE...,1,Rewrite the following tweets 5 times to genera...,[Calling jumbo shrimp on a menu what it isn't ...
3,"""tomorrow at 9, I'll be fangirling over the tv...",2,Rewrite the following tweets 5 times to genera...,[I'll be fangirling over the TV show 'Styled T...
4,sa-ANNE ka pupunta bukas? CURTIS video be the ...,1,Rewrite the following tweets 5 times to genera...,[Will you be heading to SOLTURA tomorrow? CURT...


In [25]:
export = base_cru_1p.copy()
export = export.explode('generated').reset_index(drop=True)


In [26]:
export.head()

,texto,label,prompt,generated
0,Really excited about Apple Watches collab with...,2,Rewrite the following tweets 5 times to genera...,I'm thrilled about the Apple Watch collaborati...
1,Really excited about Apple Watches collab with...,2,Rewrite the following tweets 5 times to genera...,I'm beyond excited about the Apple Watch Herme...
2,Really excited about Apple Watches collab with...,2,Rewrite the following tweets 5 times to genera...,I'm really looking forward to the Apple Watch ...
3,Really excited about Apple Watches collab with...,2,Rewrite the following tweets 5 times to genera...,I'm super excited about the Apple Watch Hermes...
4,Really excited about Apple Watches collab with...,2,Rewrite the following tweets 5 times to genera...,I'm incredibly excited about the Apple Watch H...


In [27]:
export.shape

(1020, 4)

In [28]:
base_cru_1p.shape

(210, 4)

In [29]:
export.shape[0]/base_cru_1p.shape[0]

4.857142857142857

In [30]:
arquivo = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/gptparaphrase/base_tweet_sentiment_'+str(perc)+'_gptparaphrase'+str(i)+'.pkl'
export[['texto', 'label', 'generated']].to_pickle(arquivo)